In [16]:
import tensorflow as tf
import time

#### 1. 生成文件存储样例数据。

In [17]:
# 将int64转换成tf.train.Feature格式
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# 定义将数据写入多少个文件
num_shards = 2 

# 定义每个文件写入多少个样本
instances_per_shard = 2 

for i in range(num_shards):
    filename = ('../../datasets/data.tfrecords-%.5d-of-%.5d' % (i, num_shards)) 
    
    # 将Example结构写入TFRecord文件。
    writer = tf.python_io.TFRecordWriter(filename)
    
    for j in range(instances_per_shard):
        
        # Example结构仅包含当前样例属于第几个文件以及是当前文件的第几个样本。
        example = tf.train.Example(features=tf.train.Features(feature={
            'i': _int64_feature(i),
            'j': _int64_feature(j)}))
        writer.write(example.SerializeToString())
    writer.close()  

#### 2. 读取文件。

In [18]:
# 使用tf.train.match_filenames_once函数获取一个符合正则表达式的所有文件
files = tf.train.match_filenames_once("../../datasets/data.tfrecords-*")

# 使用tf.string_input_producer 产生文件名队列
filename_queue = tf.train.string_input_producer(files, shuffle=False, num_epochs = None) 

# 使用tf.TFRecordReader去读文件名队列
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)

# 解析一个样例
features = tf.parse_single_example(
      serialized_example,
      features={
          'i': tf.FixedLenFeature([], tf.int64),
          'j': tf.FixedLenFeature([], tf.int64),
      })

with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    print(sess.run(files))
    
    # 使用tf.train.Coordinator和tf.train.start_queue_runners管理和启动线程
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for i in range(6):
        print(sess.run([features['i'], features['j']]))
    # 使用coord.request_stop停止所有线程
    coord.request_stop()
    coord.join(threads)

[b'..\\..\\datasets\\data.tfrecords-00000-of-00002'
 b'..\\..\\datasets\\data.tfrecords-00001-of-00002']
[0, 0]
[0, 1]
[1, 0]
[1, 1]
[0, 0]
[0, 1]


#### 3. 组合训练数据（Batching）

In [22]:
example, label = features['i'], features['j']

# 一个batch中包含的样例数目
batch_size = 3

# 组合样例的队列中最多可以存储的样例个数
capacity = 1000 + 3 * batch_size

# 使用tf.train.batch函数组合样例
example_batch, label_batch = tf.train.batch([example, label], batch_size=batch_size, capacity=capacity)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        
    for i in range(2):
        cur_example_batch, cur_label_batch = sess.run([example_batch, label_batch])
        print(cur_example_batch, cur_label_batch)
        
    coord.request_stop()
    coord.join(threads)


[1 0 0] [0 1 0]
[0 0 1] [0 0 0]


example: 0, lable: 0

example: 0, lable: 1

example: 1, lable: 0

example: 1, lable: 1

#### 4. 使用tf.train.shuffle_batch

In [20]:
# 使用tf.train.match_filenames_once函数获取一个符合正则表达式的所有文件
files = tf.train.match_filenames_once("../../datasets/data.tfrecords-*")

# 使用tf.string_input_producer 产生文件名队列
filename_queue = tf.train.string_input_producer(files, shuffle=False, num_epochs = None) 

# 使用tf.TFRecordReader去读文件名队列
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)

# 解析一个样例
features = tf.parse_single_example(
      serialized_example,
      features={
          'i': tf.FixedLenFeature([], tf.int64),
          'j': tf.FixedLenFeature([], tf.int64),
      })

###############################################组合batch#########################
example, label = features['i'], features['j']

# 一个batch中包含的样例数目
batch_size = 4

# 组合样例的队列中最多可以存储的样例个数
capacity = 1000 + 3 * batch_size

# 使用tf.train.shuffle_batch函数组合样例,min_after_dequeue参数限制了出队时队列中元素的最少个数
example_batch, label_batch = tf.train.shuffle_batch([example, label], batch_size=batch_size, capacity=capacity, 
                                                   min_after_dequeue = 30)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    for i in range(2):
        cur_example_batch, cur_label_batch = sess.run([example_batch, label_batch])
        print(cur_example_batch, cur_label_batch)
        
    coord.request_stop()
    coord.join(threads)

[1 1 0 0] [1 0 0 1]
[1 1 0 0] [1 1 0 1]


min_after_dequeue参数虽然限制了出队时队列中元素的最少个数，但是并不是队列中元素不足min_after_dequeuecan设置的值就不出队